In [40]:
import json

import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from exceptions import NoItemsError, AttributeExtractionError
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import random

In [25]:
schedule = (1, 2)

In [7]:
with open('agents.json', 'r', encoding='utf-8') as file:
    agents = json.load(file)

agents['USER_AGENTS']

['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36']

In [ ]:
user_agent = agents['USER_AGENTS']

def scrape_site_for_links(url, item_selector):
    '''
    Simple scraper that gets a complete URL with the selected interests
    '''
    options = Options()
    options.add_argument("--headless")
    options.add_argument(f"user-agent={user_agent}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    link_list = []

    try:
        driver.get(url)
        time.sleep(random.uniform(schedule[0], schedule[1]))
        items = driver.find_elements(By.CSS_SELECTOR, item_selector)
        if not items:
            raise NoItemsError(f"No items found on {url}")
        
        for element in items:
            a_tag = element.find_element(By.TAG_NAME, "a")
            href = a_tag.get_attribute("href")
            if href:
                link_list.append(href)
            else:
                raise AttributeExtractionError(f"No href in element on {url}")
            
    except Exception as e:
        raise e
    
    finally:
        driver.quit()
        
    return link_list


def read_page_for_content(url, item_selector):
    '''
    
    '''
    options = Options()
    options.add_argument("--headless")
    options.add_argument(f"user-agent={user_agent}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    text_list = []

    try:
        driver.get(url)
        time.sleep(random.uniform(schedule[0], schedule[1]))
        sections = driver.find_elements(By.CSS_SELECTOR, item_selector)

        if not sections:
            raise NoItemsError(f"No sections found on {url}")

        for section in sections:
            h2_tag = section.find_element(By.TAG_NAME, "h2")
            if h2_tag:
                text_list.append(h2_tag.text.strip())
            p_tags = section.find_elements(By.TAG_NAME, "p")
            for p in p_tags:
                text_list.append(p.text.strip())

    except Exception as e:
        raise e
    
    finally:
        driver.quit()
        
    return text_list



In [36]:
read_page_for_content('https://career.avito.com/vacancies/data-science/16990/', "section.vacancies-detail__description")

['О команде',
 'Avito Ads — это молодая и быстро развивающаяся рекламная система внутри Авито. Мы разделяем основную миссию Авито: помочь каждому улучшить собственную жизнь и жизнь людей вокруг. Перед нами стоит амбициозная задача кратного роста в течение нескольких лет. Команда Data Science Avito Ads занимается задачами ML в области рекламных технологий: от обучения моделей ранжирования до оптимизации стратегий показа и ставок.',
 'Вам предстоит:',
 '—    разрабатывать ML-модели для автобидинга;',
 '—    разрабатывать модели для умного распределения рекламных бюджетов с целью максимизации целевых действий и/или удержания средней ставки за целевое действие;',
 '—    применять ML для сквозного ранжирования;',
 '—    использовать ML-методы для совместного ранжирования рекламной и органической выдачи Авито;',
 '—    проектировать сложные AB-эксперименты;',
 '—    совместно с командой аналитики проектировать и анализировать эксперименты как на стороне пользователей, так и на стороне реклам

In [27]:
scrape_site_for_links("https://career.avito.com/vacancies/data-science/", "div.vacancies-section__item")

['https://career.avito.com/vacancies/data-science/16990/',
 'https://career.avito.com/vacancies/data-science/16860/',
 'https://career.avito.com/vacancies/data-science/17145/',
 'https://career.avito.com/vacancies/data-science/16833/',
 'https://career.avito.com/vacancies/data-science/17058/',
 'https://career.avito.com/vacancies/data-science/17055/',
 'https://career.avito.com/vacancies/data-science/16815/',
 'https://career.avito.com/vacancies/data-science/16791/',
 'https://career.avito.com/vacancies/data-science/16793/',
 'https://career.avito.com/vacancies/data-science/16882/',
 'https://career.avito.com/vacancies/data-science/16794/',
 'https://career.avito.com/vacancies/data-science/16792/',
 'https://career.avito.com/vacancies/data-science/16700/',
 'https://career.avito.com/vacancies/data-science/16782/',
 'https://career.avito.com/vacancies/data-science/15691/',
 'https://career.avito.com/vacancies/data-science/16698/',
 'https://career.avito.com/vacancies/data-science/16277/

In [ ]:
def scrape_site_for_links_emulate_clicks(url, target_vac, menu_selector, item_selector):
    '''
    url: str - link to the site
    target_vac: str - prof area selection
    menu_selector: str - menu class
    item_selector: str - individual vacancy class
    '''

    options = Options()

    options.add_argument("--headless")
    options.add_argument("window-size=1920,1080")

    options.add_argument(f"user-agent={user_agent}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    link_list = []

    try:
        driver.get(url)
        time.sleep(2)

        menu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, menu_selector)))
        menu.click()
        time.sleep(1)

        option = wait.until(EC.element_to_be_clickable((By.XPATH, f"//div[contains(text(), '{target_vac}')]")))
        driver.execute_script("arguments[0].scrollIntoView(true);", option)
        option.click()
        time.sleep(3)

        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(random.uniform(1, 2))
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        items = driver.find_elements(By.CSS_SELECTOR, item_selector)
        for item in items:
            try:
                a_tag = item.find_element(By.TAG_NAME, "a")
                href = a_tag.get_attribute("href")
                if href:
                    link_list.append(href)
            except:
                continue

    finally:
        driver.quit()

    return link_list


In [46]:
if __name__ == "__main__":
    urls = scrape_site_for_links_emulate_clicks(
        url="https://rabota.sber.ru/search/",
        target_vac="IT: Data Science и Data Engineering",
        menu_selector='[data-testid="profarea-filter-header"]',
        item_selector='div[class*="styled__Card"]'  # ловим по маске имени класса
    )
    print(urls)

['https://rabota.sber.ru/search/4453599', 'https://rabota.sber.ru/search/4453969', 'https://rabota.sber.ru/search/4454341', 'https://rabota.sber.ru/search/4433949', 'https://rabota.sber.ru/search/4453634', 'https://rabota.sber.ru/search/4451793', 'https://rabota.sber.ru/search/4453922', 'https://rabota.sber.ru/search/4425198', 'https://rabota.sber.ru/search/4437566', 'https://rabota.sber.ru/search/4453971', 'https://rabota.sber.ru/search/4454484', 'https://rabota.sber.ru/search/4452384', 'https://rabota.sber.ru/search/4454352', 'https://rabota.sber.ru/search/4452709', 'https://rabota.sber.ru/search/4454339', 'https://rabota.sber.ru/search/4444522', 'https://rabota.sber.ru/search/4454112', 'https://rabota.sber.ru/search/4454095', 'https://rabota.sber.ru/search/4454096', 'https://rabota.sber.ru/search/4452698', 'https://rabota.sber.ru/search/4452695', 'https://rabota.sber.ru/search/4443200', 'https://rabota.sber.ru/search/4448994', 'https://rabota.sber.ru/search/4395416', 'https://rabota

In [ ]:
def scrape_site_for_links_emulate_clicks(url, target_vac, menu_selector, item_selector):
    """
    url: str - ссылка на сайт
    target_vac: str - текст направления (например "IT: Data Science и Data Engineering")
    menu_selector: str - селектор кнопки открытия меню
    item_selector: str - селектор карточек вакансий (div, внутри a)
    """

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument(f"user-agent={user_agent}")

    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    link_list = []

    try:
        driver.get(url)
        time.sleep(2)

        menu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, menu_selector)))
        menu.click()
        time.sleep(1)

        option = wait.until(EC.element_to_be_clickable((By.XPATH, f"//div[contains(text(), '{target_vac}')]")))
        driver.execute_script("arguments[0].scrollIntoView(true);", option)
        option.click()
        time.sleep(3)

        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(random.uniform(1, 2))
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        items = driver.find_elements(By.CSS_SELECTOR, item_selector)
        for item in items:
            try:
                a_tag = item.find_element(By.TAG_NAME, "a")
                href = a_tag.get_attribute("href")
                if href:
                    link_list.append(href)
            except:
                continue

    finally:
        driver.quit()

    return link_list
